# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 61862.89it/s]


In [5]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [6]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [7]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [8]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[-7.4030e-01, -2.2425e-01, -1.3538e+00,  ...,  1.1576e-01,
           9.3618e-01,  8.3588e-01],
         [-8.7616e-01, -1.6564e-01,  7.6842e-01,  ..., -3.9596e-01,
           1.3460e+00, -1.1531e+00],
         [ 6.8036e-01, -5.8503e-01, -1.1536e+00,  ..., -1.6724e+00,
          -7.4470e-01,  9.5158e-01],
         ...,
         [ 8.7331e-01, -2.2447e+00,  1.5299e+00,  ...,  1.0030e-03,
          -3.3851e-01,  4.8882e-01],
         [ 8.7331e-01, -2.2447e+00,  1.5299e+00,  ...,  1.0030e-03,
          -3.3851e-01,  4.8882e-01],
         [ 8.7331e-01, -2.2447e+00,  1.5299e+00,  ...,  1.0030e-03,
          -3.3851e-01,  4.8882e-01]],

        [[-8.3087e-01, -1.0621e+00,  1.8359e+00,  ..., -4.9555e-01,
          -1.0726e+00, -3.4788e-01],
         [ 9.3795e-01, -1.3214e+00, -1.6135e-01,  ..., -1.5055e+00,
           8.4032e-03, -1.5041e+00],
         [-2.9202e-01,  3.2602e-01, -4.7028e-01,  ..., -4.6722e-01,
          -2.0455e+00,  1.2131e+00],
         ...,
         [ 8.7331e-01, -2

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [9]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [10]:
w_0 = nn.Linear(d_model, d_model)

In [11]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [12]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [13]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [14]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0464, 0.0435, 0.0620,  ..., 0.0449, 0.0449, 0.0449],
          [0.0606, 0.0403, 0.0247,  ..., 0.0402, 0.0402, 0.0402],
          [0.0308, 0.0306, 0.0444,  ..., 0.0335, 0.0335, 0.0335],
          ...,
          [0.0371, 0.0447, 0.0505,  ..., 0.0461, 0.0461, 0.0461],
          [0.0371, 0.0447, 0.0505,  ..., 0.0461, 0.0461, 0.0461],
          [0.0371, 0.0447, 0.0505,  ..., 0.0461, 0.0461, 0.0461]],

         [[0.0438, 0.0585, 0.0423,  ..., 0.0263, 0.0263, 0.0263],
          [0.0650, 0.0533, 0.0920,  ..., 0.0339, 0.0339, 0.0339],
          [0.0522, 0.0552, 0.0349,  ..., 0.0364, 0.0364, 0.0364],
          ...,
          [0.0699, 0.0366, 0.0527,  ..., 0.0413, 0.0413, 0.0413],
          [0.0699, 0.0366, 0.0527,  ..., 0.0413, 0.0413, 0.0413],
          [0.0699, 0.0366, 0.0527,  ..., 0.0413, 0.0413, 0.0413]],

         [[0.0435, 0.0422, 0.0433,  ..., 0.0691, 0.0691, 0.0691],
          [0.0370, 0.0463, 0.0338,  ..., 0.0470, 0.0470, 0.0470],
          [0.0432, 0.0368, 0.0524,  ..., 0

In [15]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [16]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [17]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[-0.0425, -0.1513, -0.0322,  ...,  0.0210, -0.0773, -0.0627],
         [-0.0367, -0.2024,  0.0317,  ...,  0.0479, -0.0735, -0.0836],
         [-0.0208, -0.1433,  0.0025,  ...,  0.0107, -0.0604, -0.1004],
         ...,
         [-0.0639, -0.1969, -0.0218,  ...,  0.0422, -0.0249, -0.0367],
         [-0.0639, -0.1969, -0.0218,  ...,  0.0422, -0.0249, -0.0367],
         [-0.0639, -0.1969, -0.0218,  ...,  0.0422, -0.0249, -0.0367]],

        [[ 0.1218, -0.2129, -0.0539,  ...,  0.0314, -0.0605, -0.0611],
         [ 0.0503, -0.2200, -0.0483,  ...,  0.0445, -0.0394, -0.0813],
         [ 0.1081, -0.2482, -0.0596,  ...,  0.0088, -0.0886, -0.0648],
         ...,
         [ 0.1175, -0.2221, -0.0478,  ...,  0.0287, -0.0427, -0.0798],
         [ 0.1175, -0.2221, -0.0478,  ...,  0.0287, -0.0427, -0.0798],
         [ 0.1175, -0.2221, -0.0478,  ...,  0.0287, -0.0427, -0.0798]],

        [[ 0.0757, -0.1550,  0.0830,  ...,  0.0266, -0.0216,  0.0205],
         [ 0.0947, -0.2277,  0.0217,  ...,  0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [18]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = self.w_q(q)  # (B, L, d_model)
    k = self.w_k(k)  # (B, L, d_model)
    v = self.w_v(v)  # (B, L, d_model)
 

    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    
    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [19]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [20]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[ 1.9788e-02,  3.6366e-02,  1.0439e-02,  ..., -1.9316e-02,
           5.2268e-02,  9.0272e-03],
         [ 4.1677e-02,  5.1601e-02, -1.5555e-02,  ...,  2.2124e-02,
           5.1019e-02,  4.3039e-02],
         [ 2.6515e-02,  1.3386e-01, -2.7943e-03,  ..., -3.6344e-02,
           4.1472e-02,  4.1080e-02],
         ...,
         [-2.0826e-02,  7.9785e-02,  3.5761e-02,  ..., -3.4929e-03,
           2.9025e-02, -1.4477e-02],
         [-2.0826e-02,  7.9785e-02,  3.5761e-02,  ..., -3.4929e-03,
           2.9025e-02, -1.4477e-02],
         [-2.0826e-02,  7.9785e-02,  3.5761e-02,  ..., -3.4929e-03,
           2.9025e-02, -1.4477e-02]],

        [[ 4.2514e-01, -1.1362e-02, -2.8477e-01,  ..., -2.4952e-01,
           1.2188e-01,  2.0767e-01],
         [ 4.5847e-01, -2.5576e-02, -2.7866e-01,  ..., -2.6058e-01,
           1.1262e-01,  2.4419e-01],
         [ 4.4624e-01, -8.9506e-02, -3.4682e-01,  ..., -3.2138e-01,
           1.0517e-01,  1.6000e-01],
         ...,
         [ 3.7834e-01, -2